# Predict Future Sales Competition

## Description
This challenge serves as final project for the "How to win a data science competition" Coursera course.

In this competition you will work with a challenging time-series dataset consisting of daily sales data, kindly provided by one of the largest Russian software firms - 1C Company. 

We are asking you to predict total sales for every product and store in the next month. By solving this competition you will be able to apply and enhance your data science skills.

Competition Information found at [Kaggle](https://www.kaggle.com/c/competitive-data-science-predict-future-sales)

In [94]:
# Importing Libraries and allowing plots to appear in Jupyter

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

## File Descriptions

* **sales_train.csv** - the training set. Daily historical data from January 2013 to October 2015.
* **test.csv** - the test set. You need to forecast the sales for these shops and products for November 2015.
* **sample_submission.csv** - a sample submission file in the correct format.
* **items.csv** - supplemental information about the items/products.
* **item_categories.csv**  - supplemental information about the items categories.
* **shops.csv** - supplemental information about the shops.

## Data Fields
* **ID** - an Id that represents a (Shop, Item) tuple within the test set
* **shop_id** - unique identifier of a shop
* **item_id** - unique identifier of a product
* **item_category_id** - unique identifier of item category
* **item_cnt_day** - number of products sold. You are predicting a monthly amount of this measure
* **item_price** - current price of an item
* **date** - date in format dd/mm/yyyy
* **date_block_num** - a consecutive month number, used for convenience. January 2013 is 0, February 2013 is 1,..., October 2015 is 33
* **item_name** - name of item
* **shop_name** - name of shop
* **item_category_name** - name of item category

In [90]:
# import datasets 

sales = pd.read_csv('data/sales_train.csv')
items = pd.read_csv('data/items.csv')
categories = pd.read_csv('data/item_categories.csv')
shops = pd.read_csv('data/shops.csv')

In [95]:
sales.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,02.01.2013,0,59,22154,999.00,1.0
1,03.01.2013,0,25,2552,899.00,1.0
2,05.01.2013,0,25,2552,899.00,-1.0
3,06.01.2013,0,25,2554,1709.05,1.0
4,15.01.2013,0,25,2555,1099.00,1.0


In [96]:
sales.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2935849 entries, 0 to 2935848
Data columns (total 6 columns):
date              object
date_block_num    int64
shop_id           int64
item_id           int64
item_price        float64
item_cnt_day      float64
dtypes: float64(2), int64(3), object(1)
memory usage: 134.4+ MB


In [54]:
items.head()

,item_name,item_id,item_category_id
0,! ВО ВЛАСТИ НАВАЖДЕНИЯ (ПЛАСТ.) D,0,40
1,!ABBYY FineReader 12 Professional Edition Full...,1,76
2,***В ЛУЧАХ СЛАВЫ (UNV) D,2,40
3,***ГОЛУБАЯ ВОЛНА (Univ) D,3,40
4,***КОРОБКА (СТЕКЛО) D,4,40


In [55]:
categories.head()

,item_category_name,item_category_id
0,PC - Гарнитуры/Наушники,0
1,Аксессуары - PS2,1
2,Аксессуары - PS3,2
3,Аксессуары - PS4,3
4,Аксессуары - PSP,4


In [56]:
shops.head()

,shop_name,shop_id
0,"!Якутск Орджоникидзе, 56 фран",0
1,"!Якутск ТЦ ""Центральный"" фран",1
2,"Адыгея ТЦ ""Мега""",2
3,"Балашиха ТРК ""Октябрь-Киномир""",3
4,"Волжский ТЦ ""Волга Молл""",4


In [52]:
items.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22170 entries, 0 to 22169
Data columns (total 3 columns):
item_name           22170 non-null object
item_id             22170 non-null int64
item_category_id    22170 non-null int64
dtypes: int64(2), object(1)
memory usage: 519.7+ KB


In [53]:
shops.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60 entries, 0 to 59
Data columns (total 2 columns):
shop_name    60 non-null object
shop_id      60 non-null int64
dtypes: int64(1), object(1)
memory usage: 1.0+ KB


In [64]:
''' 
Join all tables together to make master table with all info
Start with transactional table (sales) and bring in reference tables
(data is in star schema)

'''
master = sales.merge(
    items, 
    how = 'left', 
    on = 'item_id').merge(
        categories, 
        how = 'left', 
        on = 'item_category_id').merge(
            shops, 
            how = 'left', 
            on = 'shop_id')
master.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day,item_name,item_category_id,item_category_name,shop_name
0,02.01.2013,0,59,22154,999.00,1.0,ЯВЛЕНИЕ 2012 (BD),37,Кино - Blu-Ray,"Ярославль ТЦ ""Альтаир"""
1,03.01.2013,0,25,2552,899.00,1.0,DEEP PURPLE The House Of Blue Light LP,58,Музыка - Винил,"Москва ТРК ""Атриум"""
2,05.01.2013,0,25,2552,899.00,-1.0,DEEP PURPLE The House Of Blue Light LP,58,Музыка - Винил,"Москва ТРК ""Атриум"""
3,06.01.2013,0,25,2554,1709.05,1.0,DEEP PURPLE Who Do You Think We Are LP,58,Музыка - Винил,"Москва ТРК ""Атриум"""
4,15.01.2013,0,25,2555,1099.00,1.0,DEEP PURPLE 30 Very Best Of 2CD (Фирм.),56,Музыка - CD фирменного производства,"Москва ТРК ""Атриум"""


In [66]:
'''
Cell to check if there are any null values
There are no null values so no fixing to worry about
'''
master.isnull().any().any()

False

In [69]:
# getting rid of redundant columns
master = master[['shop_name', 
                 'item_category_name', 
                 'item_name', 
                 'item_price', 
                 'item_cnt_day', 
                 'date', 
                 'date_block_num']]
master.head()

,shop_name,item_category_name,item_name,item_price,item_cnt_day,date,date_block_num
0,"Ярославль ТЦ ""Альтаир""",Кино - Blu-Ray,ЯВЛЕНИЕ 2012 (BD),999.00,1.0,02.01.2013,0
1,"Москва ТРК ""Атриум""",Музыка - Винил,DEEP PURPLE The House Of Blue Light LP,899.00,1.0,03.01.2013,0
2,"Москва ТРК ""Атриум""",Музыка - Винил,DEEP PURPLE The House Of Blue Light LP,899.00,-1.0,05.01.2013,0
3,"Москва ТРК ""Атриум""",Музыка - Винил,DEEP PURPLE Who Do You Think We Are LP,1709.05,1.0,06.01.2013,0
4,"Москва ТРК ""Атриум""",Музыка - CD фирменного производства,DEEP PURPLE 30 Very Best Of 2CD (Фирм.),1099.00,1.0,15.01.2013,0


In [103]:
# filtering the master dataset to only include items which had negative sales 
negative = master[['shop_name', 'item_category_name', 'item_cnt_day', 'item_name']]
negative = negative[negative['item_cnt_day'] < 0]
#negative[negative['item_cnt_day'] < -1].groupby(['shop_name', 'item_category_name']).sum()#, 'item_name']).sum()
negative.groupby('item_name').sum().sort_values('item_cnt_day')

,item_cnt_day
item_name,
"Call of Duty: Ghosts [PS4, русская версия]",-60.0
"Grand Theft Auto V [Xbox 360, русские субтитры]",-45.0
"Grand Theft Auto V [PS3, русские субтитры]",-34.0
"Bestseller. S.T.A.L.K.E.R. [PC, Jewel]",-33.0
PS4: Контроллер игровой беспроводной черный (Dualshock 4 Cont Anthracite Black: CUH-ZCT1: SCEE),-31.0
"Одни из нас [PS3, русская версия]",-28.0
"LEGO Batman 2: DC Super Heroes [PС, Jewel, русские субтитры]",-27.0
Kinect Star Wars [только для Kinect] [Xbox 360],-27.0
Sony PlayStation 4 (500 Gb) Black (CUH-1008A/1108A/B01),-26.0
